In [2]:
import json

In [6]:
result_defected = json.load(open('/data/gunsbrother/repos/vq2d-lightning/outputs/batch/2024-11-03/137206/test_predictions.json'))

In [19]:
count = 0
pred_tree = {}
for pred_video in result_defected['results']['videos']:
    video_uid = pred_video['video_uid']
    pred_tree[video_uid] = {}
    for pred_clip in pred_video['clips']:
        clip_uid = pred_clip['clip_uid']
        pred_tree[video_uid][clip_uid] = {}
        for pred_qsets in pred_clip['predictions']:
            if 'annotation_uid' not in pred_qsets:
                pred_tree[video_uid][clip_uid] = None
                continue
            annotation_uid = pred_qsets['annotation_uid']
            pred_tree[video_uid][clip_uid][annotation_uid] = {}
            for qset_id, pred_qset in pred_qsets['query_sets'].items():
                pred_tree[video_uid][clip_uid][annotation_uid][qset_id] = pred_qset
                count += 1

count

4461

In [21]:
result = {
    'version': '1.0.5',
    'challenge': 'ego4d_vq2d_challenge',
    'results': {
        'videos': [
        ]
    }
}

count = 0
anns = json.load(open('/data/datasets/ego4d_data/v2/annotations/vq_test_unannotated.json'))
for ann_videos in anns['videos']:
    video_uid = ann_videos['video_uid']
    result['results']['videos'].append({
        'video_uid': video_uid,
        'clips': []
    })
    for ann_clips in ann_videos['clips']:
        clip_uid = ann_clips['clip_uid']
        result['results']['videos'][-1]['clips'].append({
            'clip_uid': clip_uid,
            'predictions': []
        })
        for ann_annots in ann_clips['annotations']:
            annotation_uid = ann_annots['annotation_uid']
            result['results']['videos'][-1]['clips'][-1]['predictions'].append({'query_sets': {}})
            for qset_id, ann_qsets in ann_annots['query_sets'].items():
                pred_qsets = pred_tree[video_uid][clip_uid][annotation_uid]
                if qset_id not in pred_qsets:
                    continue
                result['results']['videos'][-1]['clips'][-1]['predictions'][-1]['query_sets'][qset_id] = \
                    pred_tree[video_uid][clip_uid][annotation_uid][qset_id]
                count += 1
count

TypeError: 'NoneType' object is not subscriptable